In [1]:
#
# David Laziuk
# Unfinished CART decision tree from scratch
#

In [59]:
#Imports
import math
import numpy as np
import pandas as pd

In [60]:
data=pd.read_csv('breast-cancer-wisconsin.csv',na_values= "?")
#Dropping rows with NAs
data=data.dropna()
#Dropping Sample ID Column
data=data.iloc[:,1:]
print("Original Shape: "+str(data.shape))
data.head()

Original Shape: (683, 10)


,F1,F2,F3,F4,F5,F6,F7,F8,F9,Class
0,5,1,1,1,2,1.0,3,1,1,2
1,5,4,4,5,7,10.0,3,2,1,2
2,3,1,1,1,2,2.0,3,1,1,2
3,6,8,8,1,3,4.0,3,7,1,2
4,4,1,1,3,2,1.0,3,1,1,2


In [61]:
#Shuffling Entries
data=data.sample(frac=1).reset_index(drop=True)
#70/30 split into train/test data
train_size=int(np.floor(data.shape[0]*.7))
train=data.iloc[0:train_size,:].reset_index(drop=True)
test=data.iloc[train_size:,:].reset_index(drop=True)
del train_size

In [62]:
#Node class: Contains col<=cond & destination path
class Node:
    def __init__(self,Col,Cond,Path,Leaf):
        self.Col=Col
        self.Cond=Cond
        self.Path=Path
        self.Leaf=Leaf

In [112]:
#Defining Function to get Information Gain of split
def getIG(data,colname,val):
    #Splitting based on condition
    left_data=data[data[colname]<=val]
    right_data=data[data[colname]>val]
    #Composition of class frequency
    P_comp=data["Class"].value_counts()
    L_comp=left_data["Class"].value_counts()
    R_comp=right_data["Class"].value_counts()
    
    #Calculating Probabilities
    #Very ugly solution but extracting info from value_counts series was a headache
    #Mainly b/c the series can be of size 1 or 2,
    #and the order of "2" count & "4" count will be inconsistent
    P_two_count=P_four_count=L_two_count=L_four_count=R_two_count=R_four_count=0
    for i in range(P_comp.size):
        if(P_comp.index[i]==2):
            P_two_count=P_comp.iloc[i]  
        else:
            P_four_count=P_comp.iloc[i]
    for i in range(L_comp.size):
        if(L_comp.index[i]==2):
            L_two_count=L_comp.iloc[i]  
        else:
            L_four_count=L_comp.iloc[i]
    for i in range(R_comp.size):
        if(R_comp.index[i]==2):
            R_two_count=R_comp.iloc[i]  
        else:
            R_four_count=R_comp.iloc[i]
    P_size=data.shape[0]
    L_size=left_data.shape[0]
    R_size=right_data.shape[0]
    P_two_prob=P_two_count/P_size if(P_size>0) else 0
    P_four_prob=P_four_count/P_size if(P_size>0) else 0
    L_two_prob=L_two_count/L_size if(L_size>0) else 0
    L_four_prob=L_four_count/L_size if(L_size>0) else 0
    R_two_prob=R_two_count/R_size if(R_size>0) else 0
    R_four_prob=R_four_count/R_size if(R_size>0) else 0
    #Calculating Entropies
    P_two_log=math.log(P_two_prob,2) if(P_two_count!=0) else 0
    P_four_log=math.log(P_four_prob,2) if(P_four_count!=0) else 0
    L_two_log=math.log(L_two_prob,2) if(L_two_count!=0) else 0
    L_four_log=math.log(L_four_prob,2) if(L_four_count!=0) else 0
    R_two_log=math.log(R_two_prob,2) if(R_two_count!=0) else 0
    R_four_log=math.log(R_four_prob,2) if(R_four_count!=0) else 0
    P_ent=-((P_two_prob*P_two_log)+(P_four_prob*P_four_log))
    L_ent=-((L_two_prob*L_two_log)+(L_four_prob*L_four_log))
    R_ent=-((R_two_prob*R_two_log)+(R_four_prob*R_four_log))
    #Calculating information gain
    Split_ent=(L_ent*(L_size/P_size))+(R_ent*(R_size/P_size))
    IG=P_ent-Split_ent
    #Determining most frequent class in Left&Right split
    if(L_two_prob>L_four_prob):
        L_class=2
    else:
        L_class=4
    if(R_two_prob>R_four_prob):
        R_class=2
    else:
        R_class=4
    #Returning IG, most freq. class&entropy of Left&Right
    return IG,L_ent,L_class,R_ent,R_class,L_size,R_size,L_two_count,L_four_count,R_two_count,R_four_count

In [115]:
#Defining Function to grow tree
def growTree(data,path):
    # "B" prefix for best
    B_IG=0
    B_colname=""
    B_cond=0
    B_L_ent=-1
    B_L_class=-1
    B_R_ent=-1
    B_R_class=-1
    B_L_size=0
    B_R_size=0
    BLtc=BLfc=BRtc=BRfc=0
    #loop through F1-F10
    for f in range(9):
        colname="F"+str(f+1)
        #loop through possible value splits 2-9 (1&10 is no split b/c cond. is <=)
        for v in range(2,10):
            IG,L_ent,L_class,R_ent,R_class,L_size,R_size,BLtc,BLfc,BRtc,BRfc=getIG(data,colname,v)
            #Store info with best IG
            if(IG>B_IG):
                B_IG=IG
                B_colname=colname
                B_cond=v
                B_L_ent=L_ent
                B_L_class=L_class
                B_R_ent=R_ent
                B_R_class=R_class
                B_L_size=L_size
                B_R_size=R_size
    print("Path: "+path)
    print(B_colname+"<="+str(B_cond)+" IG: "+str(round(B_IG,4)))
    print("Left Entropy: "+str(round(B_L_ent,4))+"  Best Class: "+
          str(B_L_class)+"  Size: "+str(B_L_size)+"  2s': "+str(BLtc)+"  4s': "+str(BLfc))
    print("Right Entropy: "+str(round(B_R_ent,4))+"  Best Class: "+
          str(B_R_class)+"  Size: "+str(B_R_size)+"  2s': "+str(BRtc)+"  4s': "+str(BRfc))
    
    #Adding Node with colname<=cond & path
    nodes.append(Node(B_colname,B_cond,path,None))
    
    #Continuing Tree
    #Splitting Data on best condition
    left_split=data[data[B_colname]<=B_cond]
    right_split=data[data[B_colname]>B_cond]
    #Left
    L_path=path+"l"
    #If left entropy=0, add leaf with best left class & end branch
    if(B_L_ent==0):
        nodes.append(Node(None,None,L_path,B_L_class))
    #Else continue growing branch
    else:
        growTree(left_split,L_path)
    #Right
    R_path=path+"r"
    #If right entropy=0, add leaf with best left class & end branch
    if(B_R_ent==0):
        nodes.append(Node(None,None,R_path,B_R_class))
    #Else continue growing branch
    else:
        growTree(right_split,R_path)    

In [116]:
#Training
nodes=[]
growTree(train,"")

Path: 
F2<=2 IG: 0.5851
Left Entropy: 0.1632  Best Class: 2  Size: 292  2s': 314  4s': 153
Right Entropy: 0.6244  Best Class: 4  Size: 186  2s': 0  4s': 11
Path: l
F6<=5 IG: 0.1043
Left Entropy: 0.0599  Best Class: 2  Size: 287  2s': 285  4s': 7
Right Entropy: -0.0  Best Class: 4  Size: 5  2s': 0  4s': 0
Path: ll
F6<=2 IG: 0.0303
Left Entropy: -0.0  Best Class: 2  Size: 272  2s': 285  4s': 2
Right Entropy: 0.5665  Best Class: 2  Size: 15  2s': 0  4s': 0
Path: llr
F1<=3 IG: 0.2429
Left Entropy: -0.0  Best Class: 2  Size: 10  2s': 13  4s': 2
Right Entropy: 0.971  Best Class: 2  Size: 5  2s': 0  4s': 0
Path: llrr
F1<=5 IG: 0.3219
Left Entropy: 0.8113  Best Class: 2  Size: 4  2s': 3  4s': 2
Right Entropy: -0.0  Best Class: 4  Size: 1  2s': 0  4s': 0
Path: llrrl
F4<=2 IG: 0.3113
Left Entropy: -0.0  Best Class: 2  Size: 2  2s': 3  4s': 1
Right Entropy: 1.0  Best Class: 4  Size: 2  2s': 0  4s': 0
Path: llrrlr
F1<=4 IG: 1.0
Left Entropy: -0.0  Best Class: 4  Size: 1  2s': 1  4s': 1
Right Entro

In [102]:
#Visualizing
for i in range(len(nodes)):
    if(nodes[i].Leaf==None):
        print(str(nodes[i].Path)+" "+str(nodes[i].Col)+"<="+str(nodes[i].Cond))
    else:
        print(str(nodes[i].Path)+" "+str(nodes[i].Leaf))

 F2<=2
l F6<=5
ll F6<=2
lll 4
llr F1<=3
llrl 4
llrr F1<=5
llrrl F4<=2
llrrll 4
llrrlr F1<=4
llrrlrl 4
llrrlrr 4
llrrr 4
lr 4
r F2<=4
rl F6<=2
rll F8<=2
rlll 4
rllr F4<=3
rllrl F1<=5
rllrll 4
rllrlr F3<=4
rllrlrl 4
rllrlrr 4
rllrr 4
rlr F1<=6
rlrl F7<=3
rlrll F2<=3
rlrlll F4<=4
rlrllll 4
rlrlllr 4
rlrllr 4
rlrlr F6<=5
rlrlrl F5<=3
rlrlrll 4
rlrlrlr 4
rlrlrr 4
rlrr 4
rr F7<=4
rrl F1<=6
rrll F2<=6
rrlll 4
rrllr F2<=9
rrllrl 4
rrllrr 4
rrlr 4
rrr 4


In [103]:
#Testing
count=0
#Looping through all rows in test
for r in range(test.shape[0]):
    truth=test.loc[r,'Class']
    pred=0
    path=""
    #Navigating Tree
    while(pred==0):
        loc=-1
        #Searching Tree for matching node
        for i in range(len(nodes)):
            if(nodes[i].Path==path):
                loc=i
                break
        #Testing if node is leaf
        if(nodes[i].Leaf==None):
            #print(str(nodes[loc].Path)+" "+str(test.loc[r,nodes[loc].Col])+"<="+str(nodes[loc].Cond))
            #Evaluating Condition
            if(test.loc[r,nodes[loc].Col]<=nodes[loc].Cond):
                path+="l"
            else:
                path+="r"
        else:
            pred=nodes[i].Leaf
            #print("Pred "+str(pred)+" Truth: "+str(truth))
            if(pred==truth):
                count+=1
print("Accuracy: "+str(count/test.shape[0]))

Accuracy: 0.36585365853658536
